In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [3]:
dataset=pd.read_csv('Churn_Modelling.csv')
X=dataset.iloc[:,3:-1].values #indexing starts from 0 in python and the first three(0,1,2) columns are useless hence we are starting from 3 and at the end we are skipping the last column(-1) because that is our target variable
y=dataset.iloc[:,-1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


In [6]:
#taking care of missing data
from sklearn.impute import SimpleImputer
# Example: select only numeric columns
num_cols = dataset.select_dtypes(include=np.number).columns
dataset[num_cols] = SimpleImputer(strategy='mean').fit_transform(dataset[num_cols])

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

In [8]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [10]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [11]:
#splitting dataset into training and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [14]:
#BUILDING ANN

#initializing the ann
ann=tf.keras.models.Sequential()

In [16]:
#adding input and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [17]:
#adding second hidden output layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [18]:
#adding output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [20]:
#TRAINING THE ANN

#compiling the ANN
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
#training the ANN on the training set
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.8633 - loss: 0.3287
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.8627 - loss: 0.3285
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8630 - loss: 0.3287  
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.8615 - loss: 0.3283
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.8639 - loss: 0.3281
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.8620 - loss: 0.3286
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.8624 - loss: 0.3283
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.8618 - loss: 0.3281
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.8627 - loss: 0.3284
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8618 - loss: 0.3277 
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.8615 - loss: 0.3285
Epoch 12/100
250/250

In [31]:
#predicting the result of single observation
# print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[False]]


In [33]:
#predicting the test set results
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [45]:
#making the confusion matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(accuracy_score(y_test,y_pred)*100,'%')

[[1500   95]
 [ 180  225]]
86.25 %
